In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
covid_df = pd.read_csv('ny_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train, covid_test = covid_df[0:train_size], covid_df[train_size:]

In [3]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [4]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_1[1].to_list())

['new_cases_10_mean', 'new_cases_10_min', 'new_cases_7_lag', 'new_cases_1 lag', 'new_cases_3_lag', 'minimum_temperature_celsius', 'new_cases_10_max', 'booster_rate', 'Administered_Dose1_Recip_5Plus', 'new_cases_2_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_hospitalized_patients', 'new_cases_4_lag', 'Booster_Doses_18Plus', 'new_cases_50_max', 'vaccination_rate_complete', 'Series_Complete_18Plus', 'new_cases_10_std', 'Administered_Dose1_Recip_18Plus', 'new_cases_50_min', 'Administered_Dose1_Recip_65Plus', 'Series_Complete_5Plus', 'new_cases_100_std', 'new_cases_150_lag', 'new_cases_50_lag', 'Booster_Doses_50Plus', 'vaccination_rate_1dose', 'new_cases_50_std', 'dew_point', 'new_cases_100_max', 'new_cases_30_lag', 'new_cases_350_lag', 'Administered_Dose1_Recip_12Plus', 'new_deaths_2_lag', 'Series_Complete_65Plus', 'new_deaths_50_lag', 'Booster_Doses_65Plus', 'Series_Complete_12Plus', 'relative_humidity', 'Series_Complete_Yes']


In [5]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_2[1].to_list())

['new_cases_1 lag', 'new_cases_7_lag', 'new_cases_3_lag', 'new_cases_10_std', 'new_cases_4_lag', 'new_cases_350_lag', 'new_cases_10_mean', 'new_cases_2_lag', 'relative_humidity', 'new_cases_10_max', 'new_cases_30_lag', 'new_hospitalized_patients', 'new_cases_100_std', 'new_cases_10_min', 'new_cases_150_lag', 'maximum_temperature_celsius', 'dew_point', 'new_deaths_10_std', 'rainfall_mm', 'new_cases_50_mean', 'current_intensive_care_patients', 'new_cases_50_std', 'new_deaths_7_lag', 'new_deaths_1_lag', 'minimum_temperature_celsius', 'average_temperature_celsius', 'new_deaths_3_lag', 'new_deaths_50_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_50_lag', 'new_deaths_2_lag', 'new_deaths_100_mean', 'new_deaths_10_mean', 'new_deaths_4_lag', 'new_deaths_30_lag', 'Administered_Dose1_Recip', 'new_deaths_50_lag', 'new_cases_100_mean', 'new_deaths_100_std', 'Administered_Dose1_Recip_5Plus']


In [6]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_3[1].to_list())

['new_cases_10_max', 'new_cases_10_mean', 'new_cases_1 lag', 'new_cases_7_lag', 'new_cases_50_std', 'new_cases_350_lag', 'new_cases_10_std', 'new_cases_2_lag', 'new_deaths_1_lag', 'PEOPLE_DEATH_NEW_COUNT', 'minimum_temperature_celsius', 'new_hospitalized_patients', 'new_cases_150_lag', 'new_cases_30_lag', 'new_deaths_50_lag', 'new_deaths_30_lag', 'new_deaths_50_mean', 'average_temperature_celsius', 'maximum_temperature_celsius', 'new_cases_3_lag', 'new_cases_4_lag', 'new_deaths_10_mean', 'new_deaths_7_lag', 'dew_point', 'new_cases_100_mean', 'new_deaths_100_mean', 'new_deaths_10_std', 'new_cases_10_min', 'new_deaths_100_std', 'rainfall_mm', 'current_intensive_care_patients', 'new_deaths_4_lag', 'Administered_Dose1_Recip_18Plus', 'new_cases_100_std', 'relative_humidity', 'new_deaths_50_min', 'new_deaths_3_lag', 'Series_Complete_5Plus', 'new_deaths_2_lag', 'Series_Complete_18Plus']


In [7]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_4[1].to_list())

['new_cases_10_mean', 'new_cases_50_std', 'new_cases_10_max', 'new_cases_10_min', 'new_cases_3_lag', 'Booster_Doses_18Plus', 'new_cases_1 lag', 'Series_Complete_12Plus', 'Administered_Dose1_Recip_5Plus', 'new_cases_350_lag', 'minimum_temperature_celsius', 'new_cases_2_lag', 'new_cases_7_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_hospitalized_patients', 'new_cases_50_lag', 'dew_point', 'new_deaths_1_lag', 'relative_humidity', 'new_deaths_3_lag', 'new_cases_10_std', 'new_cases_150_lag', 'rainfall_mm', 'new_deaths_50_lag', 'new_cases_4_lag', 'new_deaths_10_mean', 'Series_Complete_Yes', 'vaccination_rate_complete', 'new_cases_30_lag', 'new_deaths_2_lag', 'Series_Complete_65Plus', 'new_deaths_50_std', 'Administered_Dose1_Recip', 'new_cases_100_mean', 'Administered_Dose1_Recip_12Plus', 'new_deaths_10_std', 'booster_rate', 'new_deaths_30_lag', 'Booster_Doses_65Plus', 'average_temperature_celsius']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(40)
print(predictors_5[1].to_list())

['new_cases_10_mean', 'new_cases_1 lag', 'new_cases_10_max', 'new_cases_50_std', 'Booster_Doses_65Plus', 'new_cases_3_lag', 'new_cases_350_lag', 'new_cases_10_min', 'new_cases_2_lag', 'new_cases_10_std', 'new_cases_7_lag', 'Booster_Doses', 'booster_rate', 'new_cases_100_std', 'Administered_Dose1_Recip_65Plus', 'Administered_Dose1_Recip', 'Series_Complete_12Plus', 'Series_Complete_5Plus', 'Administered_Dose1_Recip_5Plus', 'Booster_Doses_50Plus', 'Series_Complete_Yes', 'relative_humidity', 'new_deaths_1_lag', 'PEOPLE_DEATH_NEW_COUNT', 'Series_Complete_18Plus', 'vaccination_rate_1dose', 'vaccination_rate_complete', 'Booster_Doses_18Plus', 'new_cases_150_lag', 'Administered_Dose1_Recip_12Plus', 'new_hospitalized_patients', 'new_deaths_10_std', 'Series_Complete_65Plus', 'minimum_temperature_celsius', 'Administered_Dose1_Recip_18Plus', 'new_cases_30_lag', 'new_deaths_30_lag', 'new_cases_50_lag', 'new_deaths_3_lag', 'new_deaths_4_lag']


In [9]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
--------------------------------------------------------
Basis Function                     Pruned  Coefficient  
--------------------------------------------------------
(Intercept)                        No      8776         
h(new_cases_10_mean-9816.9)        No      8.5099       
h(9816.9-new_cases_10_mean)        No      -2.432       
h(new_cases_1 lag-10319)           No      -1.47548     
h(10319-new_cases_1 lag)           No      -0.150382    
h(new_cases_2_lag-10074)           No      -1.50665     
h(10074-new_cases_2_lag)           No      0.208379     
h(new_cases_50_mean-7090.86)       No      -1.27292     
h(7090.86-new_cases_50_mean)       No      0.691411     
h(new_cases_3_lag-6051)            No      -1.30049     
h(6051-new_cases_3_lag)            No      1.49935      
h(new_hospitalized_patients-1083)  No      -29.1863     
h(1083-new_hospitalized_patients)  No      -1.43724     
h(new_cases_10_mean-6680)          Yes     None         
h(6680-new_cases_10

In [10]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [11]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [12]:
full = common_member(a, b, c, d, e)
full

{'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_mean', 'new_cases_350_lag', 'new_cases_1 lag', 'new_cases_150_lag', 'new_cases_2_lag', 'new_cases_30_lag', 'minimum_temperature_celsius', 'new_cases_10_std', 'new_hospitalized_patients', 'new_cases_10_max', 'new_cases_7_lag', 'new_cases_10_min', 'relative_humidity', 'new_cases_3_lag', 'new_cases_50_std'}
